In [ ]:
import pandas as pd
from pymongo import MongoClient
import re
from haversine import haversine
import googlemaps
from datetime import datetime
import os
from dotenv import load_dotenv
load_dotenv()
import requests, json 
import time
client = MongoClient()

# Bring documents from Mongodb

In [2]:
def connectCollection(database, collection):
    db = client[database]
    coll = db[collection]
    return db, coll

In [3]:
db, coll = connectCollection('companies','companies')

In [4]:
compas = list(coll.find({"$and":[{"deadpooled_year":None},{"founded_year":{"$gte":1999}},{"$where":"this.offices.length >= 1"},
         {"$or":[{"offices.latitude":{'$ne':None}},{"offices.longitude":{'$ne':None}}]}]}))

# Filtering and new collections

In [5]:
db, coll1 = connectCollection('companies','airports')
db, coll2 = connectCollection('companies','preferred')

In [6]:
airports = list(coll1.find({"type": 'large_airport'}))

In [7]:
business = list(coll2.find({"$and":[{"money_raised":{"$ne":"$0"}},{"year":{"$gte":2009}}]}))

Create new collections:

In [ ]:
'''
myclient = MongoClient("mongodb://localhost/")
mydb = myclient["companies"]
mycol = mydb["final5"]
'''

To import a csv to Mongodb:

In [22]:
'''
mongoimport --db companies --collection kindergarten --type csv --file kindergarten.csv --headerline
'''

'\nmongoimport --db companies --collection kindergarten --type csv --file kindergarten.csv --headerline\n'

# Harversine 

Since the Harversine function take a couple of coordinates and return a distance, it helped to conclude which of the 99 companies have an airport closer. 

These 99 companies are the preferred after applying the different filters mentioned before. 

In [8]:
def getcoord(query,lat,long):
    for que in query: 
        lat.append(que['location']['coordinates'][1])
        long.append(que['location']['coordinates'][0])
    return len(lat),len(long)       

In [9]:
name=[]
for bus in business:
    name.append(bus['name'])

In [10]:
lat1=[]
long1=[]
getcoord(business,lat1,long1)

(99, 99)

In [11]:
bus=list(zip(lat1,long1))

In [12]:
lat2=[]
long2=[]
getcoord(airports,lat2,long2)

(614, 614)

In [13]:
air=list(zip(lat2,long2))

In [14]:
def harver(elem,zipi):
    comparator=[]
    for a in air:
        comparator.append(haversine(elem,a))
    return min(comparator)

In [15]:
futcol=[]
for i in bus:
    futcol.append(harver(i,air))

In [16]:
dfair=pd.DataFrame({"name":name,"long":long1,"lat":lat1,"dis to an airport":futcol})

In [17]:
actuali=list(zip(business,futcol))

In [19]:
for i in actuali:
    value={"$set":{'distance':i[1]}}
    coll1.update_one(i[0],value)

In [21]:
cutoffs = [0,10,20,50,100,300]
km_labels = ['0-10km', '10-20km', '20-50km','50-100km','100-300km']
dfair['range'] = pd.cut(dfair['dis to an airport'], cutoffs, labels = km_labels)
dfair['range'].value_counts()

10-20km      43
0-10km       26
20-50km      24
50-100km      4
100-300km     2
Name: range, dtype: int64

# Google API

In [ ]:
def nearbyAPI(location, radius, keyword):
        url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        places = []
        params = {
            'location': location,
            'radius': radius,
            'keyword': keyword,
            'key': os.getenv("API_KEY")
        }
        res = requests.get(url, params = params)
        results =  json.loads(res.content)
        places.extend(results['results'])
        while "next_page_token" in results:
            params['pagetoken'] = results['next_page_token'],
            res = requests.get(url, params = params)
            results = json.loads(res.content)
            places.extend(results['results'])
        return places

In [ ]:
def getcoord(query, lat, long):
    for que in query: 
        lat.append(que['location']['coordinates'][1])
        long.append(que['location']['coordinates'][0])
    return len(lat),len(long)  

In [ ]:
lat=[]
long=[]
getcoord(business,lat,long)

In [ ]:
lista=[]
busines=list(zip(lat,long))

In [ ]:
for i in busines:
    lista.append("{},{}".format(i[0],i[1]))

### API request

In [ ]:
start=time.time()
results=[]
for i in lista:    
    results.append(nearbyAPI(i, "2000", "nightclub"))
end=time.time()
print(end - start)

In [ ]:
def createApiDf(query):
    name=[]
    long=[]
    lat=[]
    for location in query:
        for starbucks in location:
            name.append(starbucks['name'])
            long.append(starbucks['geometry']['location']['lng'])
            lat.append(starbucks['geometry']['location']['lat'])
    dic2={"name":name,"long":long,"lat":lat}
    lil_DF=pd.DataFrame(dic2)
    return lil_DF 

In [ ]:
df_molona = createApiDf(results)

Be aware of dropping those entries which the API return duplicated.  

In [ ]:
df_molona=df_molona.drop_duplicates()

In [ ]:
#export_csv = df_molona.to_csv('nightclubs99.csv', index=None, header=True)

# Get Coord

In [ ]:
pref_comp = list(coll.find())

In [ ]:
def getApiLoc(off):
    longitude = off['long']
    latitude = off['lat']
    loc=({
        'type':'Point',
        'coordinates':[longitude,latitude]
    })
    return loc

In [ ]:
for pref in pref_comp:
    value = {"$set": {'location':getApiLoc(pref)}}
    coll.update_one(pref,value)

# Geoqueries

In [ ]:
db, coll3 = connectCollection('companies','starbucks')

In [ ]:
star= list(coll3.find())

In [ ]:
business = list(coll2.find({"$and":[{"money_raised":{"$ne":"$0"}},{"year":{"$gte":2009}}]}))

In [ ]:
def geoquery(geoindex,distance,col):
    points= coll.find(
        {"location": 
         {"$near": 
          {"$geometry":
           geoindex,
           "$maxDistance":distance
            }
        }
    })
    return list(points)
#que la función me devuelva la len(list(points))

In [ ]:
comparatorDosPuntoCero=[]
for bu in business:
    comparatorDosPuntoCero.append(geoquery(bu['location'],100,coll))

In [ ]:
cuantos=[]
for i in comparatorDosPuntoCero:
    cuantos.append(len(i))
data["starbucks 100m"]=cuantos

In [ ]:
#export_csv = data.to_csv('master99.csv', index=None, header=True)

In [ ]:
data = pd.read_csv("master99.csv") 
data.head()